<a href="https://colab.research.google.com/github/sganguly3000ai/makemore/blob/main/makemore_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bigram character level language model using neural network MLP**

In [98]:
import torch
import numpy as np
import string
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [119]:

#f = open('sample_data/yob1996.txt', encoding="utf-8")
names   = np.loadtxt('sample_data/yob1996.txt', delimiter=',', usecols = 0, dtype = str)
lnames  = np.char.lower(names)
unames  = np.unique(lnames)
# for l in f:
#   print (l)


In [118]:
print(f'{names} : {lnames}')

['Emily' 'Jessica' 'Ashley' ... 'Zishe' 'Zoran' 'Zyler'] : ['emily' 'jessica' 'ashley' ... 'zishe' 'zoran' 'zyler']


In [120]:
names.shape,lnames.shape,unames.shape

((26426,), (26426,), (24306,))

In [111]:
names1.shape

AttributeError: 'set' object has no attribute 'shape'

In [ ]:

p = Path('./sample_data')


# create a list of words or names
words = ['sumit','sorit','swapna','suraj','sujit', 'sujoy', 'manoj', 'amit',
         'kajol', 'steve', 'andrew','bob', 'cindy', 'ebrahim', 'daku', 'johny',
         'timmy', 'david', 'tarun', 'nancy', 'edith', 'jason', 'kyle', 'joseph',
         'kimberly', 'emma', 'irene', 'michael', 'scott', 'valentino']
# build the vocabulary of characters and mappings to/from integers
# '.' is the special character signifying the start or end of a word
chars = list(string.ascii_lowercase)
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
print(f'Num of words = {len(words)}')
print(f'Vocabulary = {itos}')

Num of words = 30
Vocabulary = {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [],[]
num_of_words = 0
for w in words:

  #print(w)
  context = [0] * block_size

  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix] #crop and append
num_of_words += 1

X = torch.tensor(X)
Y = torch.tensor(Y)

num_of_labels = len(Y)


In [ ]:
# make it more respectable and lets put the code together

In [ ]:
X.shape, Y.shape # datasets (inputs and labels)

(torch.Size([188, 3]), torch.Size([188]))

In [ ]:
g   =   torch.Generator().manual_seed(2147483647)
C   =   torch.randn((27,2), generator = g)
W1  =   torch.randn((6,100), generator = g)
b1  =   torch.randn((100), generator = g)
W2  =   torch.randn((100,27), generator = g)      # 27 possible outputs for 27 lowercase english characters
b2  =   torch.randn((27), generator = g)
parameters  = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters)             # number of parameters in total

3481

In [ ]:
for p in parameters:
  p.requires_grad = True

In [ ]:
# to figure out what learning rate to use
# guess a learning rate (one low and one high)
# plug in the (guessed) low learning rate and train the model - observe the rate at which the loss is going down
# plug in the (guessed) high learning rate and train the model - observe the rate at which the loss is going down
# the (guessed) low and high lr should both not make the loss go down fairly quickly
# for this particular model it seems like the low and high should be .001 and 1

lre = torch.linspace(-3,0,1000)   # create evenly spaced values between -3 and 0
lrs = 10**lre                     # taking the exponent gives us a tensor of values in the range 1 and .001

# now instead of using the guessed learning rate (lr), we will run the model with each learning rate in lrs
# and track the losses for a 1000 iterations of training the model

In [ ]:
# lri   = []    # used to track the learning rate
# lossi = []    # used to track the losses for the given lr

for i in range(10000):

  # minibatch construct (pluck 31 samples out of the total number of samples randomly

  ix = torch.randint(0, X.shape[0], (31,))


  # forward pass
  emb     = C[X[ix]]                              # shape is 31, 3, 2
  h       = torch.tanh(emb.view(-1,6) @ W1 + b1)  # shape is 31, 100
  logits  = h @ W2 + b2                           # shape is 31, 27 (27 possible outputs for 27 lowercase english characters)
  # counts  = logits.exp()
  # probs   = counts / counts.sum(1, keepdims = True)
  # loss    = -probs[torch.arange(num_of_labels),Y].log().mean()
  loss = F.cross_entropy(logits, Y[ix])

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  # gradient update
  for p in parameters:
    #lr        = lrs[i]
    p.data   += -0.1 * p.grad

"""
    p.data    += -lr * p.grad

  # track stats for lr
  lri.append(lre[i])
  lossi.append(loss.item())
"""


print(loss.item())

0.43690159916877747


In [ ]:
plt.plot(lri,lossi) # looks like 10**-1, which is .1 is a good learning rate

In [ ]:
# train, validation and test split
# should not run the test data through the model too many times. if you do that the model will start to train on the test data

trainsplt = int(len(X)*.8)
vsplt     = int(len(X)*.1)
testsplt  = int(len(X)) - trainsplt - vsplt
xtrain, xv, xtest = [],[],[]
xtrain, xv, xtest = torch.split(X, (trainsplt,vsplt,testsplt))
ytrain, yv, ytest = [],[],[]
ytrain, yv, ytest = torch.split(Y, (trainsplt,vsplt,testsplt))

In [ ]:

print(f'{ytrain}\n{ytrain.shape}\n{yv}\n{yv.shape}\n{ytest}\n{ytest.shape}')

tensor([19, 21, 13,  9, 20,  0, 19, 15, 18,  9, 20,  0, 19, 23,  1, 16, 14,  1,
         0, 19, 21, 18,  1, 10,  0, 19, 21, 10,  9, 20,  0, 19, 21, 10, 15, 25,
         0, 13,  1, 14, 15, 10,  0,  1, 13,  9, 20,  0, 11,  1, 10, 15, 12,  0,
        19, 20,  5, 22,  5,  0,  1, 14,  4, 18,  5, 23,  0,  2, 15,  2,  0,  3,
         9, 14,  4, 25,  0,  5,  2, 18,  1,  8,  9, 13,  0,  4,  1, 11, 21,  0,
        10, 15,  8, 14, 25,  0, 20,  9, 13, 13, 25,  0,  4,  1, 22,  9,  4,  0,
        20,  1, 18, 21, 14,  0, 14,  1, 14,  3, 25,  0,  5,  4,  9, 20,  8,  0,
        10,  1, 19, 15, 14,  0, 11, 25, 12,  5,  0, 10, 15, 19,  5, 16,  8,  0,
        11,  9, 13,  2,  5, 18])
torch.Size([150])
tensor([12, 25,  0,  5, 13, 13,  1,  0,  9, 18,  5, 14,  5,  0, 13,  9,  3,  8])
torch.Size([18])
tensor([ 1,  5, 12,  0, 19,  3, 15, 20, 20,  0, 22,  1, 12,  5, 14, 20,  9, 14,
        15,  0])
torch.Size([20])


In [ ]:
v[0],X[10]

(tensor([15, 18,  9]), tensor([15, 18,  9]))